# Arbori de decizie. Păduri aleatoare

* Tudor Berariu - 2016
* George Muraru - 2020

## 1. Scopul laboratorului

Scopul laboratorului îl reprezintă întelegerea conceptului de arbore de decizie și implementarea unor clasificatori bazați pe acest model.

## 2. Problema de rezolvat

Problema de rezolvat ı̂n acest laborator este una de ı̂nvățare supervizată: fiind dat un **set de date X** ce conține exemple descrise printr-un set de **atribute discrete A** și etichetate cu **câte o clasă dintr-o mulțime cunoscută C**, să se construiască un model pentru clasificarea exemplelor noi.

## 3. Arbore de decizie


Un arbore de decizie este un clasificator ce aproximează funcții discrete.

Într-un arbore de decizie există 2 tipuri de noduri:
* *noduri intermediare* - conține un test pentru un atribut și are câte un arc (și implicit un subarbore) pentru fiecare valoare posibiliă a atributului
* *noduri frunză* - este etichetat cu o clasă

Pentru **a clasifica un obiect nou** se pornește din rădăcina arborelui și din fiecare nod se coboară pe arcul corespunzător valorii atributului pe care o are obiectul dat. Atunci când se ajunge ı̂ntr-un nod frunză, clasa acestuia va reprezenta predicția arborelui.

## 4. Păduri de arbori aleatori

*Pădurile de arbori aleatori* (eng. Random Forest) este un model format din mai mulți arbori de decizie.

Se bazează pe 2 hiperparametrii:
* Eșantionare aleatoare din setul de date de antrenament
* Subseturi aleatoare de atribute considerate la împărțirea pe mai multi subarbori

Predicția, utilizând un astfel de model, se bazează pe clasa majoritară oferită de predicțiile indepente ale tuturor arborilor.

## 5. Workspace Setup

### Câteva biblioteci de care vom avea nevoie

In [14]:
from math import log2
import csv
import os

### Hiperparametrii necesari rulării

In [15]:
CLASS = "CLASS"

DATASET_NAME = 'Car'  #@param ['Chess', 'Car', 'Tennis']

# Adâncimea arborilor
D = 3 #@param {type: "slider", min: 2, max: 10}

# Procentul de exemple din setul de date utilizat la construcția arborilor
P = 50 #@param {type: "slider", min: 1, max: 100}

### Funcții ajutătoare pentru descărcarea și lucrul cu setul de date

In [16]:
class Node:
    """ Representation for a node from the decision tree """
    def __init__(self, label):
        """
            for non-leafs it is the name of the attribute
            for leafs it is the class
        """
        self.label = label

        # Dictionary of (attribute value, nodes)
        self.children = {}

    def display(self, indent = ""):
        print(indent + (self.label + ":" if self.children else "<" + self.label + ">"))
        indent += "   "
        if self.children:
            for key, value in self.children.items():
                print(indent + ":" + key)
                value.display(indent + "   ")


def getArchive(dataSetName):
    """ Checks if a specific dataset is present in the local directory, if not,
    downloads it.

    Args:
        dataSetName (str): the dataset name
    """
    try:
        # Load the Drive helper and mount
        from google.colab import drive

        # This will prompt for authorization.
        drive.mount('/content/drive/')
    except:
        print("Cannot connect to Google Drive")

    dataset_file = "datasets" + os.sep + dataSetName.lower()
    from os import path
    if not path.isfile(dataset_file):
        print(f"Cannot find dataset {dataSetName} at {dataset_file}")
        dataset_file = "/content/drive/My Drive/Colab Notebooks/" + dataset_file
    print(dataset_file)

    from os import path
    if not path.isfile(dataset_file):
        print(f"Cannot find dataset {dataSetName} at {dataset_file}")
    else:
        print(f"{dataset_file} already in the local directory")


def getDataSet(dataSetName):
    """ Reads a dataset

    Args:
        dataSetName (str): Name for the dataset

    Returns:
        A tuple containing (classes, attributes, examples):
        classes (set): the classes that are found in the dataset
        attributes (list of strings): the attributes for the dataset
        examples (list of dictionaries): one example contains an entry as
            (attribute name, attribute value)
    """

    dataset_file = "datasets" + os.sep + dataSetName.lower()
    from os import path
    if not path.isfile(dataset_file):
        print(f"Cannot find dataset {dataSetName} at {dataset_file}")
        dataset_file = "/content/drive/My Drive/Colab Notebooks/" + dataset_file
    if not path.isfile(dataset_file):
        print(f"Cannot find dataset {dataSetName} in {dataset_file}")

    f_in = open(dataset_file, 'r')
    csv_reader = csv.reader(f_in, delimiter=",")

    # Read the header row
    row = next(csv_reader)

    # The last element represents the class
    attributeNames = row[:-1]

    examples = []
    classes = set()

    for row in csv_reader:
        *attributes, label = row
        classes.add(label)
        example = dict(zip(attributeNames, attributes))
        example[CLASS] = label
        examples.append(example)

    f_in.close()
    return classes, attributeNames, examples

### Descărcare și încarcare set de date

In [17]:
getArchive(DATASET_NAME)
classes, attributes, examples = getDataSet(DATASET_NAME)
print("Classes:", classes)
print("Attributes:", attributes)
print()
print("Examples (first examples in the dataset):")
for ex in examples[:5]: print(ex)


Cannot connect to Google Drive
datasets/car
datasets/car already in the local directory
Classes: {'good', 'vgood', 'acc', 'unacc'}
Attributes: ['buying', 'maint', 'doors', 'persons', 'lug_boot']

Examples (first examples in the dataset):
{'buying': 'vhigh', 'maint': 'vhigh', 'doors': '2', 'persons': '2', 'lug_boot': 'small', 'CLASS': 'unacc'}
{'buying': 'vhigh', 'maint': 'vhigh', 'doors': '2', 'persons': '2', 'lug_boot': 'small', 'CLASS': 'unacc'}
{'buying': 'vhigh', 'maint': 'vhigh', 'doors': '2', 'persons': '2', 'lug_boot': 'small', 'CLASS': 'unacc'}
{'buying': 'vhigh', 'maint': 'vhigh', 'doors': '2', 'persons': '2', 'lug_boot': 'med', 'CLASS': 'unacc'}
{'buying': 'vhigh', 'maint': 'vhigh', 'doors': '2', 'persons': '2', 'lug_boot': 'med', 'CLASS': 'unacc'}


1. [3 pct] Implementați o funcție recursivă *randomTree* care construiește arbori de decizie de adâncime **d** pe baza unui **set de date X** și a unei **mulțimi de atribute A** astfel:
 * Dac̆a *d = 0* sau nu mai sunt atribute rămase, atunci se construiește un nod frunză cu clasa majoritară din X.
 * Dacă *d > 0*, atunci se alege aleator un atribut $a_i$ din A și se construiește câte un subarbore pentru fiecare valoare $v_j$ a atributului $a_i$ apelând *randomTree* pentru *d − 1*:
$$
X_{i/j} = \{x \in X|a_{i}(x) = v_j\}\\
A_{new} = A \setminus \{a_i\}
$$

## 6. Cerințe

In [61]:
from collections import Counter
from copy import deepcopy
import random
from typing import Dict


def mostFrequentClass(X):
    # TODO 1a
    cnt = Counter([ex["CLASS"] for ex in X])
    return cnt.most_common(1)[0][0]

def randomTree(d, X, A):
    """
    Args:
        d: maximum depth
        X: a list of examples
        A: the list of attribute names
    """
    # TODO 1b

    if d == 0 or len(A) == 0:
        return Node(mostFrequentClass(X))
    # if  or len(X) == 0:
        # return Node("")
    Acopy = deepcopy(A)
    att = random.choice(Acopy)
    Acopy.remove(att)
    used_values = {}
    new_x = []
    node = Node(att)
    # print(A)
    # print(X)
    for x in X:
        value = x[att]

        if value in used_values:
            continue
        used_values[value] = True
    # print(used_values)
    for v1 in used_values:
        new_x = []

        for x in X:
            if x[att] == v1:
                # print(l)
                to_add = deepcopy(x)
                to_add.pop(att)
                new_x.append(to_add)
        node.children[v1] = randomTree(d - 1, new_x, Acopy)
    return node

## pentru datasets/car, pentru d = 5, ar trebui să existe și unele frunze care nu sunt etichetate cu "unacc"
# verificați pentru d mai mare decât numărul de atribute
print("Tree:")
# randomTree(2, examples, attributes).display()
tree = randomTree(5, deepcopy(examples), deepcopy(attributes))
print(tree.children)
tree.display()
# randomTree(10, examples, attributes).display()

Tree:
{'vhigh': <__main__.Node object at 0x7f2060f3d270>, 'high': <__main__.Node object at 0x7f2061210160>, 'med': <__main__.Node object at 0x7f206368e590>, 'low': <__main__.Node object at 0x7f206114e7a0>}
buying:
   :vhigh
      doors:
         :2
            lug_boot:
               :small
                  maint:
                     :vhigh
                        persons:
                           :2
                              <unacc>
                           :4
                              <unacc>
                           :more
                              <unacc>
                     :high
                        persons:
                           :2
                              <unacc>
                           :4
                              <unacc>
                           :more
                              <unacc>
                     :med
                        persons:
                           :2
                              <unacc>
                    

2. [3 pct] Implementați o funcție recursivă *id3* care construiește arbori de decizie pe baza unui **set de date X** și a unei **mulțimi de atribute A**.
    
  Trebuie avute în vedere următoarele aspecte:
  * dacă toate exemplele din X aparțin unei singure clase C, atunci se construiește un nod frunză etichetat cu acea clasă C
  * dacă nu mai exista atribute, atunci construiește nodul frunză etichetat cu cea mai frecventă clasă din X
    
  În caz contrar:
  * se alege atributul $a^*$ care aduce cel mai mai mare câștig informațional și se construiește un *nod intermediar* corespunzător acestuia.

  $$
    entropy(X) = -\sum_{c \in C}\frac{|X_c|}{|X|}log_2\frac{|X_c|}{|X|}
  $$
  $$
    gain(X, a_i) = entropy(X) - \sum_{v_{j} \in vals(a_i)} \frac{|X_{i/j}|}{|X|}entropy(X_{i/j})
  $$
  $$
    a^* = \underset{a_i \in A}{\operatorname{arg max}}\ gain(X, a_i)
  $$

  * pentru fiecare valoare posibilă $v_j$ a lui $a^*$ se construiește un subarbore apelând recursiv funcția *id3* pentru:

$$
X_{i/j} = \{x \in X|a_{i}(x) = v_j\}\\
A_{new} = A \setminus \{a_i\}
$$

În cazul prezentat mai sus, entropia este utilizată pentru a măsura randomness-ul din date. Intuitiv, cu cât un eveniment are probabilitate mai mare să se întâmple atunci acesta va avea o entropia din ce în ce mai mică. Prin modul în care se construiește arborele *ID3* se încearcă reducerea entropiei alegând la fiecare pas atributele care ne ofera cea mai multă informație. Cât considerați că este entropia într-un *nod frunză*?

In [62]:
import math


def entropy(X):
    entropy = 0
    total_instances = len(X)
    cnt = Counter([ex["CLASS"] for ex in X])
    for _, v in cnt.items():
        entropy += (v/total_instances) * log2(v/total_instances)
    return -entropy


def gain(X, a):
    # TODO 2b
    gain = 0
    total_entropy = entropy(X)

    attribute_values = set(entry[a] for entry in X)
    weighted_entropy = 0

    for value in attribute_values:
        subset = [entry for entry in X if entry[a] == value]
        probability = len(subset) / len(X)
        weighted_entropy += probability * entropy(subset)

    gain = total_entropy - weighted_entropy
    return gain


def id3(X, A, d = 6):
    # TODO 2c

    return Node("")

def evaluate(tree, example):
    '''
    Functia intoarce clasa prezisa de arborele `tree` pentru exemplul `example`
    '''
    # TODO 2d

    return None


def precision(tree, X, c):
    prec = 0
    predicted_ct = 0

    for ex in X:
        pred_c = evaluate(tree, ex)
        if pred_c == c:
            predicted_ct += 1
            if ex['CLASS'] ==c:
                prec += 1

    if predicted_ct != 0:
        return prec / predicted_ct
    return 0


def recall(tree, X, c):
    X_c = [x for x in X if x['CLASS'] == c]
    recall = len(list([x for x in X_c if evaluate(tree, x) == c]))
    recall /= len(X_c)
    return recall

def accuracy(tree, X):
    count = len(list(x for x in X if evaluate(tree, x) == x['CLASS']))
    return 1.0 * count / len(X)


classes, attributes, examples = getDataSet("car")
# classes, attributes, examples = getDataSet("tennis")
# classes, attributes, examples = getDataSet("chess")

print(entropy(examples)) # ~ 1.2057 pentru datasets/car
if DATASET_NAME == "Car":
    print(gain(examples, "persons")) # ~ 0.2197
    print(gain(examples, "doors")) # ~ 0.0044
    print(gain(examples, "lug_boot")) # ~ 0.03
tree = id3(examples, attributes, 3)
rtree = randomTree(3, examples, attributes)
if DATASET_NAME == "Car": print(evaluate(tree, {"maint": "med", "buying": "med", "doors": "2", "persons": "4", "lug_boot": "med"})) # acc

cls = list(classes)[-2]
print("Accuracy, precision, recall, for id3, class", cls, ":", round(accuracy(tree, examples), 3),
      round(precision(tree, examples, cls), 3), round(recall(tree, examples, cls), 3))
print("Accuracy, precision, recall, for random, class", cls, ":", round(accuracy(rtree, examples), 3),
      round(precision(rtree, examples, cls), 3), round(recall(rtree, examples, cls), 3))
print("Tree:")
tree.display()

1.2057409700121753
0.2196629633399082
0.004485716626632108
0.030008141247605424
None
Accuracy, precision, recall, for id3, class acc : 0.0 0 0.0
Accuracy, precision, recall, for random, class acc : 0.0 0 0.0
Tree:
<>


Output for `id3(examples, attributes, 3).display()` for `dataset/car`
```
persons:
   :2
      <unacc>
   :4
      buying:
           (...)
        :med
            maint:
               :vhigh
                  <unacc>
               :high
                  <unacc>
               :med
                  <acc>
               :low
                  <unacc>
         :low
            maint:
               :vhigh
                  <unacc>
               :high
                  <acc>
               :med
                  <unacc>
               :low
                  <unacc>
   :more
      buying:
             (...)
         :med
            maint:
               :vhigh
                  <unacc>
               :high
                  <unacc>
               :med
                  <acc>
               :low
                  <unacc>
         :low
            maint:
               :vhigh
                  <unacc>
               :high
                  <acc>
               :med
                  <unacc>
               :low
                  <unacc>
```
`dataset/tennis`:
```
Outlook:
   :sunny
      Humidity:
         :high
            <no>
         :normal
            <yes>
   :overcast
      <yes>
   :rain
      Windy:
         :false
            <yes>
         :true
            <no>
```

3. [4 pct] Implementați clasificator de tip pădure de arbori aleatori construind *n* arbori de adâncime maximă *d* fiecare dintre aceștia pornind de la o submulțime aleatoare a lui X.

    Folosiți funcția *randomTree* de la cerința 1.
  * Porniți de la *n = 100*, *d = 3* și submulțimi formate din 50% din elementele lui X alese la întamplare și experimentați cu acești hiperparametrii.
  * Pentru predicția clasei pentru obiecte noi alegeți clasa majoritară
  * Comparați rezultatele obținute folosind un singur arbore construit cu ID3 și o pădure de arbori aleatori. Discuție după *zgomot*, *overfitting*.

In [ ]:
def randomForest(X, n, d):
    # TODO Cerință 3
    pass

## 7. Set de date

În cadrul acestui laborator se vor folosi seturile de date [Car Evaluation](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation), [Chess](https://archive.ics.uci.edu/ml/datasets/Chess+%28King-Rook+vs.+King-Pawn%29) și [Tennis](https://www.kaggle.com/fredericobreno/play-tennis).

Aceste seturi de date sunt "ușor" modificate astfel încât pe prima linie să se afle atributele și labelul/clasa din care face parte fiecare exemplu.

Atributele datasetului *Chess* nu sunt intuitive, iar dacă doriți să aflați mai multe informații despre acestea, puteți accesa link-ul de [aici](https://pdfs.semanticscholar.org/db58/88d3f373aff2c6bd7b2f956b81c6896874a9.pdf?_ga=2.193733611.798337455.1582711694-486327444.1582711694).

## 8. Extra

### 8.1 ID3 exemplu
Un exemplu mai detaliat pentru construcția arborelui de decizie ID3 se poate găsi [aici](https://github.com/cs-pub-ro/ML/blob/master/lab/lab2/id3_example.pdf).

### 8.2 CART
Un alt algoritm utilizat poartă denumirea de CART (eng. Classification and Regression Tree). Dacă **ID3** utilizeaza **câștigul informațional (eng. information gain)**, **CART** utilizeaza o altă metrică numită **index-ul Gini (eng. Gini index sau Gini impurity)**.

Pentru implementare, se urmăresc exact aceeași [pași ca la ID3](#scrollTo=rjYqUPSbe1gG), singura diferentă fiind modul în care se calculează atributul utilizat într-un *nod intermediar*.
$$
Gini(X, a) = 1 - \sum_{c \in C}{p(c | attr(X) = a) ^2}
\\
a^* = \underset{a \in A}{\operatorname{arg min}}\ Gini(X, a)
$$